# Forecasting Net Prophet

You’re a growth analyst at [MercadoLibre](http://investor.mercadolibre.com/investor-relations). With over 200 million users, MercadoLibre is the most popular e-commerce site in Latin America. You've been tasked with analyzing the company's financial and user data in clever ways to make the company grow. So, you want to find out if the ability to predict search traffic can translate into the ability to successfully trade the stock.


## Install and import the required libraries and dependencies

In [ ]:
# Install the required libraries
!pip install pystan
!pip install prophet
!pip install hvplot
!pip install holoviews

In [ ]:
# Import the required libraries and dependencies
from google.colab import files
import warnings
import numpy as np
import pandas as pd
import holoviews as hv
from prophet import Prophet
import hvplot.pandas
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Ignoring warnings
warnings.filterwarnings('ignore')

## Step 1: Find Unusual Patterns in Hourly Google Search Traffic

The data science manager asks if the Google search traffic for the company links to any financial events at the company. Or, does the search traffic data just present random noise? To answer this question, pick out any unusual patterns in the Google search data for the company, and connect them to the corporate financial events.

To do so, complete the following steps:

1. Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

2. Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?


#### Step 1: Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

In [ ]:
# Global Variables

# Date
date = 'Date'
# Framew Widht
frame_width = 1000
# Frame Height
frame_height = 500
# Color
color = '#9AF000'
# Bakckground Color
background_color = '#eeeeee'
# Font Sizes
font_sizes = {'title': 24, 'labels': 18,'xticks': 12, 'yticks': 12}
# Mercado Libre
mercado_libre = 'MercadoLibre'
# Search Trends
search_trends = 'Search Trends'
# Lagged Search Trends
lagged_search_trends = f'Lagged {search_trends}'
# Stock Volatility
stock_volatility = 'Stock Volatility'
# Font Weight Bold
bold = 'bold'
# Figure Size
figsize = (20,10)
# Close
close = 'close'
# Stock Price
stock_price = 'Stock Price'
# Forecasted
forecasted = 'Forecasted'
# Prophet Columns
prophet_columns = ['ds','y']
# Yhat Columns
yhat_columns = ['yhat','yhat_lower','yhat_upper']

In [ ]:
# Utility lambda method to read and return DataFrame
get_dated_df = lambda title, is_lowercase=True: pd.read_csv(
  f'{title}.csv',
  index_col=date.lower() if is_lowercase else date,
  parse_dates=True,
  infer_datetime_format=True
)

In [ ]:
# Utility lambda method to display the head and tail of a DataFrame
display_head_tail = lambda df: display(df.head(),df.tail())

In [ ]:
# Upload the "google_hourly_search_trends.csv" file into Colab, then store in a Pandas DataFrame
# Set the "Date" column as the Datetime Index.
files.upload()

df_mercado_trends = get_dated_df('google_hourly_search_trends', False)

# Review the first and last five rows of the DataFrame
display_head_tail(df_mercado_trends)

In [ ]:
# Review the data types of the DataFrame using the info function
df_mercado_trends.dtypes

In [ ]:
# Utility lambda method to plot DataFrame
plot_df = lambda df, title: df.hvplot(
  title=title, 
  grid=True,
  frame_width=frame_width,
  frame_height=frame_height,
  color=color
).opts(
  bgcolor=background_color, 
  fontsize=font_sizes
)

In [ ]:
# Utility lambda method to set bokeh as the Holoviews extension
set_hv_extension_bokeh = lambda: hv.extension('bokeh')

In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# Slice the DataFrame to just the month of May 2020
df_may_2020 = df_mercado_trends.loc['2020-05']

# Use hvPlot to visualize the data for May 2020
plot_df(df_may_2020,f'{mercado_libre} {search_trends} May 2020')


**Question:** Do any unusual patterns exist?

**Answer:** On 5/04 and 5/05 the search trends spike ~20%.

#### Step 2: Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

In [ ]:
# Calculate the sum of the total search traffic for May 2020
traffic_may_2020 = df_may_2020[search_trends].sum()

# View the traffic_may_2020 value
traffic_may_2020


In [ ]:
# Calcluate the monhtly median search traffic across all months 
# Group the DataFrame by index year and then index month, chain the sum and then the median functions
median_monthly_traffic = df_mercado_trends.groupby(by=[df_mercado_trends.index.year,df_mercado_trends.index.month])[search_trends].sum().median()

# View the median_monthly_traffic value
median_monthly_traffic


In [ ]:
# Compare the seach traffic for the month of May 2020 to the overall monthly median value
traffic_may_2020 > median_monthly_traffic


##### Answer the following question: 

**Question:** Did the Google search traffic increase during the month that MercadoLibre released its financial results?

**Answer:** Yes


## Step 2: Mine the Search Traffic Data for Seasonality

Marketing realizes that they can use the hourly search data, too. If they can track and predict interest in the company and its platform for any time of day, they can focus their marketing efforts around the times that have the most traffic. This will get a greater return on investment (ROI) from their marketing budget.

To that end, you want to mine the search traffic data for predictable seasonal patterns of interest in the company. To do so, complete the following steps:

1. Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

2. Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

3. Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?


#### Step 1: Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

In [ ]:
# Utility method to get title
get_average_traffic_title = lambda timeframe: f'{mercado_libre} Average {timeframe} Traffic'

In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# Group the hourly search data to plot (use hvPlot) the average traffic by the day of week 
# Saving DataFrame
average_daily_traffic_df = df_mercado_trends.groupby(by=[df_mercado_trends.index.dayofweek]).mean()
# Plotting DataFrame
plot_df(average_daily_traffic_df, get_average_traffic_title("Daily"))

#### Step 2: Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# Use hvPlot to visualize the hour of the day and day of week search traffic as a heatmap.
df_mercado_trends.hvplot.heatmap(
  x='index.hour',
  y='index.dayofweek',
  xlabel='Hour',
  ylabel='Day of the Week',
  xticks=list(range(0,24)),
  C=search_trends,
  title= f'{mercado_libre} Hourly vs Daily {search_trends} Heatmap', 
  grid=True,
  frame_width=frame_width,
  frame_height=frame_height,
  cmap='bwr'
).aggregate(
  function=np.mean
).opts(
  bgcolor=background_color, 
  fontsize=font_sizes
)

##### Answer the following question:

**Question:** Does any day-of-week effect that you observe concentrate in just a few hours of that day?

**Answer:** Yes, the concentration is focused on hours 7-9.

#### Step 3: Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# Group the hourly search data to plot (use hvPlot) the average traffic by the week of the year
# Saving DataFrame
average_weekly_traffic_df = df_mercado_trends.groupby(by=[df_mercado_trends.index.weekofyear]).mean()
# Plotting DataFrame
plot_df(average_weekly_traffic_df,get_average_traffic_title("Weekly"))

##### Answer the following question:

**Question:** Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

**Answer:** Yes, however, not by a significant factor. For the entire period search trends only vary between 45 and 51. 

## Step 3: Relate the Search Traffic to Stock Price Patterns

You mention your work on the search traffic data during a meeting with people in the finance group at the company. They want to know if any relationship between the search data and the company stock price exists, and they ask if you can investigate.

To do so, complete the following steps:

1. Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

2. Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

3. Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

    * “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

    * “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis

4. Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?


#### Step 1: Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

In [ ]:
# Upload the "mercado_stock_price.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the Datetime Index.
files.upload()

df_mercado_stock = get_dated_df('mercado_stock_price')

# View the first and last five rows of the DataFrame
display_head_tail(df_mercado_stock)


In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# Use hvPlot to visualize the closing price of the df_mercado_stock DataFrame
plot_df(df_mercado_stock,f'{mercado_libre} {stock_price} 2015-2020')

In [ ]:
# Concatenate the df_mercado_stock DataFrame with the df_mercado_trends DataFrame
# Concatenate the DataFrame by columns (axis=1), and drop and rows with only one column of data
mercado_stock_trends_df = pd.concat([df_mercado_trends,df_mercado_stock], axis=1, join='inner').dropna()

# View the first and last five rows of the DataFrame
display_head_tail(mercado_stock_trends_df)


#### Step 2: Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

In [ ]:
# For the combined dataframe, slice to just the first half of 2020 (2020-01 through 2020-06) 
first_half_2020 = mercado_stock_trends_df.loc['2020-01':'2020-06']

# View the first and last five rows of first_half_2020 DataFrame
display_head_tail(first_half_2020)

In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# Use hvPlot to visualize the close and Search Trends data
# Plot each column on a separate axes using the following syntax
# `hvplot(shared_axes=False, subplots=True).cols(1)`
first_half_2020.hvplot(
  title=f'{mercado_libre} {search_trends} & {stock_price} January-June 2020', 
  grid=True,
  color=color,
  frame_width=frame_width,
  frame_height=frame_height,
  shared_axes=False, 
  subplots=True
).cols(1)

##### Answer the following question:

**Question:** Do both time series indicate a common trend that’s consistent with this narrative?

**Answer:** Search trends do not seem to match the increasing strock price following the events of 2020.

#### Step 3: Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

* “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

* “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis


In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Lagged Search Trends
# This column should shift the Search Trends information by one hour
mercado_stock_trends_df[lagged_search_trends] = mercado_stock_trends_df[search_trends].shift(1)

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Stock Volatility
# This column should calculate the standard deviation of the closing stock price return data over a 4 period rolling window
mercado_stock_trends_df[stock_volatility] = mercado_stock_trends_df[close].pct_change().rolling(window=4).std()


In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# Use hvPlot to visualize the stock volatility
plot_df(mercado_stock_trends_df[stock_volatility],f'{mercado_libre} {stock_volatility} 2016-2020')

**Solution Note:** Note how volatility spiked, and tended to stay high, during the first half of 2020. This is a common characteristic of volatility in stock returns worldwide: high volatility days tend to be followed by yet more high volatility days. When it rains, it pours.

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Hourly Stock Return
# This column should calculate hourly return percentage of the closing price
mercado_stock_trends_df['Hourly Stock Return'] = mercado_stock_trends_df[close].pct_change()


In [ ]:
# View the first and last five rows of the mercado_stock_trends_df DataFrame
display_head_tail(mercado_stock_trends_df)


#### Step 4: Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

In [ ]:
# Construct correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return
mercado_stock_trends_df[[stock_volatility,lagged_search_trends,'Hourly Stock Return']].corr()


##### Answer the following question:


**Question:** Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

**Answer:** There is no meaningful coorelation between any of the data series. 

## Step 4: Create a Time Series Model with Prophet

Now, you need to produce a time series model that analyzes and forecasts patterns in the hourly search data. To do so, complete the following steps:

1. Set up the Google search data for a Prophet forecasting model.

2. After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

3. Plot the individual time series components of the model to answer the following questions:

    * What time of day exhibits the greatest popularity?

    * Which day of the week gets the most search traffic?

    * What's the lowest point for search traffic in the calendar year?


#### Step 1: Set up the Google search data for a Prophet forecasting model.

In [ ]:
# Using the df_mercado_trends DataFrame, reset the index so the date information is no longer the index
mercado_prophet_df = df_mercado_trends.copy().reset_index()
# Label the columns ds and y so that the syntax is recognized by Prophet
mercado_prophet_df.columns = prophet_columns

# Drop an NaN values from the prophet_df DataFrame
mercado_prophet_df = mercado_prophet_df.dropna()

# View the first and last five rows of the mercado_prophet_df DataFrame
display_head_tail(mercado_prophet_df)

In [ ]:
# Call the Prophet function, store as an object
model_mercado_trends = Prophet()


In [ ]:
# Fit the time-series model.
model_mercado_trends.fit(mercado_prophet_df)


In [ ]:
# Create a future dataframe to hold predictions
# Make the prediction go out as far as 2000 hours (approx 80 days)
future_mercado_trends = model_mercado_trends.make_future_dataframe(periods=2000, freq='H')

# View the last five rows of the future_mercado_trends DataFrame
future_mercado_trends.tail()


In [ ]:
# Make the predictions for the trend data using the future_mercado_trends DataFrame
forecast_mercado_trends = model_mercado_trends.predict(future_mercado_trends)

# Display the first five rows of the forecast_mercado_trends DataFrame
forecast_mercado_trends.head()


#### Step 2: After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

In [ ]:
# Utility method to plot forecast
def plot_forecast(model,forecast_df,title,ylabel):
  # Plotting DataFrame
  plot = model.plot(forecast_df,figsize=figsize)

  # Styling with matplotlib.pyplot

  # Setting/Styling Title
  plt.suptitle(title, fontsize=font_sizes['title'], fontweight=bold);
  # Setting/Styling X Label
  plt.xlabel(date, fontsize=font_sizes['labels'], fontweight=bold);
  # Setting/Styling Y Label
  plt.ylabel(ylabel, fontsize=font_sizes['labels'], fontweight=bold);

In [ ]:
# Plot the Prophet predictions for the Mercado trends data
plot_forecast(model_mercado_trends,forecast_mercado_trends,f'{mercado_libre} {search_trends} {forecasted}',search_trends)

##### Answer the following question:

**Question:**  How's the near-term forecast for the popularity of MercadoLibre?

**Answer:** Near-term is trending is the negative direction.


#### Step 3: Plot the individual time series components of the model to answer the following questions:

* What time of day exhibits the greatest popularity?

* Which day of the week gets the most search traffic?

* What's the lowest point for search traffic in the calendar year?


In [ ]:
# Utility method to set index as ds
set_ds_index = lambda df: df.set_index('ds', inplace=True)

In [ ]:
# Set the index in the forecast_mercado_trends DataFrame to the ds datetime column
set_ds_index(forecast_mercado_trends)

# View the only the yhat,yhat_lower and yhat_upper columns from the DataFrame
forecast_mercado_trends[yhat_columns]


Solutions Note: `yhat` represents the most likely (average) forecast, whereas `yhat_lower` and `yhat_upper` represents the worst and best case prediction (based on what are known as 95% confidence intervals).

In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# From the forecast_mercado_trends DataFrame, use hvPlot to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 2000 hours 
forecast_mercado_trends[yhat_columns].iloc[-2000:].hvplot(
  title=f'{mercado_libre} {forecasted} Trends', 
  grid=True,
  frame_width=frame_width,
  frame_height=frame_height,
  xlabel=date,
  ylabel='Prediction',
  group_label='Y Hat',
).opts(
  bgcolor=background_color, 
  fontsize={
    'title': 24, 
    'labels': 18,
    'xticks': 12, 
    'yticks': 12,
    'legend_title': 18,
    'legend': 12
  }
)


In [ ]:
# Utility method to plot forecast
def plot_forecast_components(model,forecast,title):
  # Plotting DataFrame
  plot = model.plot_components(forecast, figsize=figsize)
  # Styling Title
  plt.suptitle(title, fontsize=font_sizes['title'], fontweight=bold);

In [ ]:
# Reset the index in the forecast_mercado_trends DataFrame
forecast_mercado_trends.reset_index(inplace=True)

# Use the plot_components function to visualize the forecast results 
plot_forecast_components(model_mercado_trends,forecast_mercado_trends,f'{mercado_libre} {forecasted} Components')

##### Answer the following questions:

**Question:** What time of day exhibits the greatest popularity?

**Answer:** 00:00:00

**Question:** Which day of week gets the most search traffic? 
   
**Answer:** Tuesday.

**Question:** What's the lowest point for search traffic in the calendar year?

**Answer:** Late October.

## Step 5 (Optional): Forecast Revenue by Using Time Series Models

A few weeks after your initial analysis, the finance group follows up to find out if you can help them solve a different problem. Your fame as a growth analyst in the company continues to grow!

Specifically, the finance group wants a forecast of the total sales for the next quarter. This will dramatically increase their ability to plan budgets and to help guide expectations for the company investors.

To do so, complete the following steps:

1. Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data. The daily sales figures are quoted in millions of USD dollars.

2. Interpret the model output to identify any seasonal patterns in the company's revenue. For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

3. Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.




#### Step 1: Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data.

In [ ]:
# Upload the "mercado_daily_revenue.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the DatetimeIndex
# Sales are quoted in millions of US dollars
files.upload()

df_mercado_sales = get_dated_df('mercado_daily_revenue')

# Review the DataFrame
df_mercado_sales


In [ ]:
# Holoviews extension to render hvPlots in Colab
set_hv_extension_bokeh()

# Use hvPlot to visualize the daily sales figures 
plot_df(df_mercado_sales,f'{mercado_libre} Daily Revenue')


In [ ]:
# Apply a Facebook Prophet model to the data.

# Set up the dataframe in the neccessary format:
# Reset the index so that date becomes a column in the DataFrame
mercado_sales_prophet_df = df_mercado_sales.copy().reset_index()

# Adjust the columns names to the Prophet syntax
mercado_sales_prophet_df.columns = prophet_columns

# Visualize the DataFrame
mercado_sales_prophet_df

In [ ]:
# Create the model
mercado_sales_prophet_model = Prophet()

# Fit the model
mercado_sales_prophet_model.fit(mercado_sales_prophet_df)

In [ ]:
# Predict sales for 90 days (1 quarter) out into the future.

# Start by making a future dataframe
mercado_sales_prophet_future = mercado_sales_prophet_model.make_future_dataframe(periods=90, freq='D')

# Display the last five rows of the future DataFrame
mercado_sales_prophet_future.tail()


In [ ]:
# Make predictions for the sales each day over the next quarter
mercado_sales_prophet_forecast = mercado_sales_prophet_model.predict(mercado_sales_prophet_future)

# Display the first 5 rows of the resulting DataFrame
mercado_sales_prophet_forecast.head()


#### Step 2: Interpret the model output to identify any seasonal patterns in the company's revenue. For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

In [ ]:
# Use the plot_components function to analyze seasonal patterns in the company's revenue
plot_forecast_components(mercado_sales_prophet_model,mercado_sales_prophet_forecast,f'{mercado_libre} Revenue {forecasted}')

##### Answer the following question:

**Question:** For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

**Answer:** Monday-Wednesday are the best revenue days, with Wednesday being the highest. 

#### Step 3: Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.

In [ ]:
# Plot the predictions for the Mercado sales
plot_forecast(mercado_sales_prophet_model,mercado_sales_prophet_forecast,f'{mercado_libre} Sales {forecasted}','Sales')


In [ ]:
# For the mercado_sales_prophet_forecast DataFrame, set the ds column as the DataFrame Index
set_ds_index(mercado_sales_prophet_forecast)

# Display the first and last five rows of the DataFrame
display_head_tail(mercado_sales_prophet_forecast)


In [ ]:
# Produce a sales forecast for the finance division
# giving them a number for expected total sales next quarter.
# Provide best case (yhat_upper), worst case (yhat_lower), and most likely (yhat) scenarios.

# Create a forecast_quarter Dataframe for the period 2020-07-01 to 2020-09-30
# The DataFrame should include the columns yhat_upper, yhat_lower, and yhat

# Generating forecast Data through 2020-09-30
forecast_quarter = mercado_sales_prophet_model.make_future_dataframe(periods=139, freq='D')
# Getting predicted forecast, setting 'ds' as index, grabbing columns yhat_upper, yhat_lower, and yhat, for the period 2020-07-01 to 2020-09-30
forecast_quarter = mercado_sales_prophet_model.predict(forecast_quarter).set_index('ds')[yhat_columns].loc['2020-07':]

# Update the column names for the forecast_quarter DataFrame
# to match what the finance division is looking for 
forecast_quarter = forecast_quarter.rename(
    columns={
      "yhat_lower":"Worst Case",                                                                             
      "yhat_upper":"Best Case",                                                                            
      "yhat":"Most Likely Case"
    }
)

# Renaming index
forecast_quarter.index.rename('Date',inplace=True)

# Review the last five rows of the DataFrame
forecast_quarter.tail()

In [ ]:
# Displayed the summed values for all the rows in the forecast_quarter DataFrame
forecast_quarter.sum()


### Based on the forecast information generated above, produce a sales forecast for the finance division, giving them a number for expected total sales next quarter. Include best and worst case scenarios, to better help the finance team plan.

**Answer:** Sales forecast is as follows:
Best Case Total Sales - 2338.97
Worst Case Total Sales - 1988.11
Most Likely Total Sales - 2163.70